In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('best_schedule.csv')

In [3]:
df

,Team1,Team2,Venue,Day,Time Slot,Week
0,Manchester United,Liverpool,Old Trafford,Thursday,16:30-18:30,1
1,Manchester United,Arsenal,Old Trafford,Friday,19:00-21:00,4
2,Liverpool,Newcastle United,Etihad Stadium,Tuesday,09:00-11:00,3
3,Manchester United,Aston Villa,Anfield,Saturday,09:00-11:00,2
4,Liverpool,Aston Villa,Stamford Bridge,Monday,09:00-11:00,1
5,Manchester City,Tottenham Hotspur,Old Trafford,Tuesday,16:30-18:30,1
6,Manchester City,Aston Villa,Anfield,Friday,11:30-13:30,1
7,Manchester United,Newcastle United,Anfield,Monday,11:30-13:30,1
8,Chelsea,Newcastle United,Etihad Stadium,Saturday,19:00-21:00,2
9,Manchester City,Newcastle United,Stamford Bridge,Saturday,14:00-16:00,3


In [4]:

teams = pd.unique(df[['Team1', 'Team2']].values.ravel())

team_matches = {}

for team in teams:
    matches = df[(df['Team1'] == team) | (df['Team2'] == team)]
    team_matches[team] = matches

# Example: show Chelsea's matches
team_matches['Chelsea']

,Team1,Team2,Venue,Day,Time Slot,Week
8,Chelsea,Newcastle United,Etihad Stadium,Saturday,19:00-21:00,2
15,Chelsea,Arsenal,Anfield,Tuesday,09:00-11:00,1
19,Chelsea,Tottenham Hotspur,Etihad Stadium,Sunday,09:00-11:00,3
21,Manchester City,Chelsea,Etihad Stadium,Saturday,16:30-18:30,4
22,Manchester United,Chelsea,Old Trafford,Tuesday,16:30-18:30,2
23,Chelsea,Aston Villa,Old Trafford,Wednesday,19:00-21:00,3
26,Liverpool,Chelsea,Anfield,Wednesday,14:00-16:00,4


In [5]:


# Day name to index
day_to_index = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2,
    "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
}


# Step 2: Create unified list of matches for each team
df_team1 = df[['Week', 'Day', 'Team1']].rename(columns={'Team1': 'Team'})
df_team2 = df[['Week', 'Day', 'Team2']].rename(columns={'Team2': 'Team'})
df_all = pd.concat([df_team1, df_team2])

# Step 3: Convert day name to absolute day count
df_all['DayIndex'] = df_all['Day'].map(day_to_index)
df_all['AbsoluteDay'] = df_all['Week'] * 7 + df_all['DayIndex']

# Step 4: Count violations per team
violation_summary = []

for team, group in df_all.groupby('Team'):
    total_matches = len(group)
    sorted_days = sorted(group['AbsoluteDay'].tolist())
    violations = 0
    for i in range(1, len(sorted_days)):
        if sorted_days[i] - sorted_days[i - 1] < 3:
            violations += 1
    violation_summary.append({
        'Team': team,
        'Matches': total_matches,
        'Violations': violations
    })

# Step 5: Show result
violation_df = pd.DataFrame(violation_summary).sort_values(by='Violations', ascending=False)
print(violation_df)


                Team  Matches  Violations
0            Arsenal        7           0
1        Aston Villa        7           0
2            Chelsea        7           0
3          Liverpool        7           0
4    Manchester City        7           0
5  Manchester United        7           0
6   Newcastle United        7           0
7  Tottenham Hotspur        7           0


In [6]:


df['Venue'] = df.apply(lambda row: (row['Venue'], row['Week'], row['Day']), axis=1)

venue_usage_counts = df['Venue'].value_counts()

venue_violations = venue_usage_counts[venue_usage_counts > 1]


total_venue_violations = sum(venue_violations - 1)


violation_details = venue_violations.reset_index()
violation_details.columns = ['(Venue, Week, Day)', 'MatchesScheduled']

print(f"Total Venue Violations: {total_venue_violations}")
print(violation_details)


Total Venue Violations: 0
Empty DataFrame
Columns: [(Venue, Week, Day), MatchesScheduled]
Index: []
